# Uploading datas

In [12]:
import pandas as pd
import numpy as np

In [2]:
basics_url = "https://datasets.imdbws.com/title.basics.tsv.gz"

In [3]:
akas_url = "https://datasets.imdbws.com/title.akas.tsv.gz"

In [4]:
rating_url = "https://datasets.imdbws.com/title.ratings.tsv.gz"

In [5]:
basics = pd.read_csv(basics_url, sep='\t', low_memory=False)

In [6]:
akas = pd.read_csv(akas_url, sep='\t', low_memory=False)

In [7]:
rating = pd.read_csv(rating_url, sep='\t', low_memory=False)

# Data Filtering/Cleaning

## Data: Title Basics

In [9]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9223790 entries, 0 to 9223789
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   tconst          object
 1   titleType       object
 2   primaryTitle    object
 3   originalTitle   object
 4   isAdult         object
 5   startYear       object
 6   endYear         object
 7   runtimeMinutes  object
 8   genres          object
dtypes: object(9)
memory usage: 633.3+ MB


In [15]:
# Replace "\N" with np.nan
basics.replace({'\\N':np.nan}, inplace=True)

In [16]:
#check missing values 
basics.isna().sum()

tconst                  0
titleType               0
primaryTitle           11
originalTitle          11
isAdult                 1
startYear         1228727
endYear           9127760
runtimeMinutes    6751395
genres             427213
dtype: int64

In [17]:
#drop missing values in runtimeminutes column
basics.dropna(subset=['runtimeMinutes'], inplace=True)

In [18]:
#drop missing values in genres column
basics.dropna(subset=['genres'], inplace=True)

In [19]:
#double check missing values
basics.isna().sum()

tconst                  0
titleType               0
primaryTitle            1
originalTitle           1
isAdult                 0
startYear           36402
endYear           2359633
runtimeMinutes          0
genres                  0
dtype: int64

In [37]:
filter1 = basics["titleType"]=="Movie"

In [38]:
filter2 = basics['startYear']>2000

In [39]:
filter3 = basics['startYear']<2022

In [40]:
basics = basics.loc[filter1&filter2&filter3]

In [43]:
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
